# Cargar Librerias

In [1]:
import matplotlib.pyplot as plt 

from tensorflow.keras.utils import img_to_array

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input

import scipy

import os

from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
from PIL import UnidentifiedImageError

import requests
from io import BytesIO
import shutil

2022-08-24 16:45:29.774954: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-24 16:45:29.820981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-24 16:45:29.821004: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


###  Arreglar imágenes

In [2]:
for folder in os.listdir('images_scrapping_17-08-22'):
    for picture in os.listdir(f'images_scrapping_17-08-22/{folder}'):
        try:
            byteImgIO = BytesIO()
            byteImg = Image.open(f'images_scrapping_17-08-22/{folder}/{picture}')
            if byteImg.mode not in ["1", "L", "P", "RGB", "RGBA"]:
                byteImg = byteImg.convert("RGB")
            byteImg.save(byteImgIO, "PNG")
            byteImgIO.seek(0)
        except IsADirectoryError:
            print("Directory"+"-"+folder +"-"+ picture)
            shutil.rmtree(f'images_scrapping_17-08-22/{folder}')
        except KeyError:
            print("Key"+"-"+folder+"-"+picture)
            continue
        except UnidentifiedImageError:
            print("Unidentified"+"-"+folder+"-"+picture)
            continue
        except FileNotFoundError :
            print("FileNotFound"+"-"+folder+"-"+picture)
            continue

FileNotFoundError: [Errno 2] No such file or directory: 'images_scrapping_17-08-22'

# Cargar Archivo 

In [3]:
train_data_dir = 'data_augmentation_images'  

In [4]:
width_shape = 224
height_shape = 224
num_classes = 2706
epochs = 10
batch_size = 32 

# Augmentation

In [5]:
train_datagen = ImageDataGenerator(  
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(  
    train_data_dir,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    save_to_dir='',
    class_mode='categorical')

Found 9544 images belonging to 2706 classes.


In [6]:
#Creación de modelo 
nb_train_samples = 9544
model = Sequential()

inputShape = (height_shape, width_shape, 3)
model.add(Conv2D(32,(3,3), input_shape=inputShape))
model.add(Conv2D(32,(3,3)))
model.add(MaxPool2D())
          
model.add(Conv2D(64,(3,3)))
model.add(Conv2D(64,(3,3)))
model.add(Conv2D(64,(3,3)))
model.add(MaxPool2D())

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(num_classes,activation='softmax', name='output'))

model.summary()

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model_history = model.fit(  
    train_generator,
    epochs=epochs,
    batch_size=batch_size)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 106, 106, 64)      36928     
                                                                 
 conv2d_4 (Conv2D)           (None, 104, 104, 64)      36928     
                                                        

2022-08-24 16:45:50.601555: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-24 16:45:50.601577: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-24 16:45:50.601590: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (1cd964e2444f): /proc/driver/nvidia/version does not exist
2022-08-24 16:45:50.601761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
 12/299 [>.............................] - ETA: 6:26 - loss: 244.6873 - accuracy: 0.0026

KeyboardInterrupt: 